In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # Кроссвалидации при обучении
from sklearn.model_selection import GridSearchCV # Для подбора лучших параметров для обучения решающего дерева
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score # Для расчета метрик качества модели
from sklearn.metrics import roc_curve, auc # Для графика рок-кривой
from sklearn.ensemble import RandomForestClassifier # Для создания рандомног леса
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

In [ ]:
Тогда я нашел довольно интересную закономерность прохождения онлайн курсов. Слушатели онлайн курсов очень негативно 
реагируют на невозможность решить задачу, иными словами, если студент застрял на определенном шаге, то он, с высокой 
вероятность, вообще бросит курс, чем просто пропустит этот шаг и продолжит обучение. 

Давайте найдем такой стэп, используя данные о сабмитах. Для каждого пользователя найдите такой шаг, который он не смог 
решить, и после этого не пытался решать другие шаги. Затем найдите id шага,  который стал финальной точкой практического 
обучения на курсе для максимального числа пользователей. 

То есть мы исследуем следующий сценарий: человек решает стэп, не может получить правильный ответ и больше не 
возвращается к практическим задачам. Что это за шаг такой, который отпугнул максимальное число пользователей?

In [ ]:
events_data_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\events_data_test.csv')
submissions_data_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\submission_data_test.csv')
events_data_train.head()
events_data_train.isnull().sum()
submissions_data_train.head()

In [ ]:
# Используя данные с пользователями которые прошли и не прошли курс
users_data.head()
gone_and_nopass_us = users_data.query('is_gone_user==True & passed_corse==False & started_attempt>0')
us_id = gone_and_nopass_us.user_id
us_id

In [ ]:
last_step = submissions_train.drop_duplicates(subset=['user_id','step_id','submission_status'])
last_time = last_step.query("user_id in @us_id").groupby('user_id',as_index=False)\
            .agg({'timestamp':'max'}).rename(columns = {'timestamp':'max_time'})
last_step = last_step.query("user_id in @us_id")
last_step = last_step.merge(last_time,how='outer')
last_step.query("timestamp==max_time & submission_status=='wrong'")\
        .groupby('step_id',as_index=False).agg({'user_id':'count'}).sort_values('user_id',ascending=False)

В этих данных, вам доступны только первые дня активности студентов для того, чтобы сделать предсказание. На самом деле, используя эти данные, вы уже можете сделать прогноз. Например, если пользователь за первые два дня набрал 40 баллов, скорее всего он наберет более 40 баллов в дальнейшем. Чтобы подкрепить такие гипотезы, вы можете использовать данные, на которые мы исследовали в первых двух модулях курса, где для всех пользователей представлены все данные об их активности на курсе. Убедитесь, что вы сформировали файл с предсказаними для всех 6184 студентов, для каждого студента должна быть предсказана вероятность, что он наберет более 40 баллов за курс. У вас есть 25 попыток засабмитить решения, в зачет пойдет наилучший вариант.

Результатом проверки этого задания будет значение ROC AUC score, именно по этому показателю мы и отберем победителей, успехов!

In [ ]:
events_data_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\events_data_test.csv')
submissions_data_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\submission_data_test.csv')
events_data_train.head()
events_data_train.isnull().sum()
submissions_data_train.head()

In [ ]:
us_min_tim_ev = events_train.groupby('user_id',as_index=False).agg({'timestamp':'min'})\
                            .rename(columns = {'timestamp':'min_timestamp'})
us_min_tim_ev.head()
us_min_tim_sub = submissions_train.groupby('user_id',as_index=False).agg({'timestamp':'min'})\
                            .rename(columns = {'timestamp':'min_timestamp'})
us_min_tim_ev.user_id.nunique()

In [ ]:
time_two = 2*24*60*60
events_train['user_time'] = events_train.user_id.map(str) + '_' +  events_train.timestamp.map(str)
us_min_tim_ev['time_thresh'] = us_min_tim_ev.user_id.map(str) + '_' + (us_min_tim_ev.min_timestamp + time_two).map(str)
us_min_tim_ev.head()

In [ ]:
events_train = events_train.merge(us_min_tim_ev[['user_id','time_thresh']],how='outer')
events_train = events_train[events_train.user_time <= events_train.time_thresh]

In [ ]:
events_train.groupby('user_id').day.nunique().max()

In [ ]:
submissions_train['user_time'] = submissions_train.user_id.map(str) + '_' +  submissions_train.timestamp.map(str)
us_min_tim_sub['time_thresh'] = us_min_tim_sub.user_id.map(str) + '_' + (us_min_tim_sub.min_timestamp + time_two).map(str)
us_min_tim_sub.head()

In [ ]:
submissions_train = submissions_train.merge(us_min_tim_sub[['user_id','time_thresh']], how = 'outer')
submissions_train = submissions_train[submissions_train.user_time <= submissions_train.time_thresh]

In [ ]:
submissions_train.groupby('user_id').day.nunique().max()

In [ ]:
users_data['full_score'] = users_data.correct >= 40
users_data.user_id.nunique()

In [ ]:
ev_two_d = events_train.pivot_table(index = 'user_id',
                                   columns = 'action',
                                   values = 'step_id',
                                   aggfunc = 'count',
                                   fill_value = 0).reset_index()
ev_two_d.head()

In [ ]:
sub_two_d = submissions_train.pivot_table(index = 'user_id',
                                   columns = 'submission_status',
                                   values = 'step_id',
                                   aggfunc = 'count',
                                   fill_value = 0).reset_index()
sub_two_d.head()

In [ ]:
df_for_tr = ev_two_d.merge(sub_two_d, how = 'outer').merge(users_data[['user_id','full_score']],how = 'outer')
df_for_tr = df_for_tr.fillna(0)
df_for_tr.head()

In [ ]:
X_train_ml = df_for_tr.drop(['user_id','full_score'],axis=1)
y_train_ml = df_for_tr.full_score

In [ ]:
us_clf = RandomForestClassifier(random_state=0)
params = {'max_depth':range(1,20), 'n_estimators':range(10,60),
         'min_samples_split':range(2,10),'min_samples_leaf':range(1,20)}
grid_search_us = RandomizedSearchCV(us_clf,params,cv=5)

In [ ]:
grid_search_us.fit(X_train_ml,y_train_ml)

In [ ]:
best_clf = grid_search_us.best_estimator_

In [ ]:
feature_importances = best_clf.feature_importances_
feature_importances_df = pd.DataFrame({'features':list(X_train_ml),
                                       'feature_importances':feature_importances,})
feature_importances_df.sort_values('feature_importances',ascending=False)

In [ ]:
submissions_data_piv = submissions_data_train.pivot_table(index = 'user_id',
                                   columns = 'submission_status',
                                   values = 'step_id',
                                   aggfunc = 'count',
                                   fill_value = 0).reset_index()
events_data_piv = events_data_train.pivot_table(index = 'user_id',
                                   columns = 'action',
                                   values = 'step_id',
                                   aggfunc = 'count',
                                   fill_value = 0).reset_index()

X_test_ml = events_data_piv.merge(submissions_data_piv, how = 'outer')
X_test_ml = X_test_ml.fillna(0)
#X_test_ml = X_test_ml.set_index('user_id')
X_test_ml.head()

In [ ]:
X_test_ml1 = X_test_ml.drop(['user_id'],axis=1)
X_test_ml1.head()

In [ ]:
predict = pd.DataFrame(best_clf.predict_proba(X_test_ml1)).iloc[:,1]
#predict = pd.DataFrame(predict).rename(columns = {1:'is_gone'})
predict.head()

In [ ]:
examp = X_test_ml[['user_id']]
examp['is_gone'] = predict
examp.head()

In [ ]:
examp.to_csv('Ml_pred.csv')

In [ ]:
sub_exampl = pd.read_csv('C:\\Users\\Ильнар\\Data science\\submission_example.csv')
sub_exampl.shape

In [ ]:
x = pd.read_csv('Ml_pred.csv',index_col=0)
x